# Azure AI Agent Service - AI Search

### *Create Azure AI Search agent to search over the documents indexed using Azure AI search service*

<span style="color:red"> Please refer *pre-requisite* section in readme file before running this notebook! </span>


#### Step 1

* Rename *credentials.txt* file to *credentials.env* file 

* Add all required configuration values in the *credentials.env* file

* Select kernel as *Python 3* in the top right corner of the notebook (Python version 3.12)



#### Step 2

* Login to Azure account, by following the below steps:

    * Right click on this notebook from left menu and open in terminal
    
    * Run the command `az login` and follow the instructions to login to your Azure account
    
    (If you see any issues logging in to Azure CLI, error might be due to azure policies, try different tenant/subscription)

In [63]:
import os
from datetime import datetime as pydatetime
from typing import Any, List, Dict
from dotenv import load_dotenv
# Azure AI Projects
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.ai.projects.models import AzureAISearchIndex
from azure.ai.agents.models import AzureAISearchTool, AzureAISearchQueryType
from azure.ai.agents.models import MessageRole, ListSortOrder

In [9]:
# Step 3 (Optional) - Not required if you are using codespaces or Jupyter notebooks with pre-installed dependencies.

# If you encounter any errors during execution of above cell, install dependencies manually by running below command:
# Make sure you are using python version > 3.10

# %pip install -r requirements.txt

Load variables from env file

In [ ]:
# load secrets
load_dotenv("credentials.env", override=True)
projectEndpoint=os.environ["PROJECT_ENDPOINT"]
modelName=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
aiSearchConnectionName = os.getenv("AISEARCH_CONNECTION_NAME")
indexName = os.getenv("AISEARCH_INDEX_NAME")

Check version of AI projects library. Make sure you are using version 1.0.0b10

In [11]:
import azure.ai.projects
print(azure.ai.projects.__version__)

1.0.0b11


In [12]:
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
project_client = AIProjectClient(endpoint=projectEndpoint, credential=credential)

In [13]:
# [START agents_sample]
agent = project_client.agents.create_agent(
    model=modelName,
    name="my-agent",
    instructions="You are helpful agent",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_ipLCTZa9BiHUTpyAUZp5MHjn


In [20]:
# List all agents to see what's already available
all_agents = project_client.agents.list_agents()

for agent in all_agents:
	print(f"- {agent.name} (ID: {agent.id})")

- my-agent (ID: asst_ipLCTZa9BiHUTpyAUZp5MHjn)


In [ ]:
print("List all connections:")
for connection in project_client.connections.list():
    print(connection)
    

In [ ]:
print("Get the default connection of a particular type, with its credentials:")
aisearch_connection = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, include_credentials=True
)
print(aisearch_connection)

------

ex-1: Enter input prompt to search over the documents indexed using Azure AI search service 
and name the agent as *AISearchWorkshopAgent*

In [23]:
newAgentName = "AISearchWorkshopAgent"
input_prompt = "does perks plus cover gym membership?"

ex-2: Create new AI search agent

In [34]:

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=aisearch_connection.id, index_name=indexName, query_type=AzureAISearchQueryType.SEMANTIC, top_k=3, filter=""
)

# Create agent with AI search tool and process assistant run

agent = project_client.agents.create_agent(
    model=modelName,
    name=newAgentName,
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources
)

ex-3: Create thread and add messages to the thread

In [ ]:
# Create thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
# Remember to update the message with your data
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=input_prompt,
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_9jAgocz3Y1w1UTD0vRiublF8
Created message, ID: msg_yCnrhEQuPDdDQHxVIhJTLGD4


ex-4: Run the agent and get the response

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
lstmsg = messages  


----

Response recvd: 

In [ ]:
for message in lstmsg:
    print(f"Role: {message.role}, Content: {message.content}")

Clean up

In [ ]:
#Delete the assistant when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
# project_client.agents.delete_thread(thread.id)
# print("Deleted thread")